# Projekt: Algorithmischer-Handel

Das Projekt Algorithmischer Handel basierend auf die Klassifikation automatisierte Handelsstrategien zu entwickeln.
Die Kauf- und Verkaufsentscheidung soll basierend auf den historischen und Echtzeit-Marktdaten funktionieren.
Die Klassifikation generiert die Vohersage, um dann zu schauen, ob die Aktie gekauft(long) oder verkauft(short) werden soll.

Das Projekt muss unter folgenden Schritte aufgeteilt werden:
1. Die Datenbeschaffung und aufbereitung
2. Datenanalyse
3. Vorbereitung der Trainingsdaten
4. Modellierung
5. Bewertimg der Genauigkeit und Zuverlässigkeit der entwickelten Methode
6. Backtesting mit Historiche Daten

# 1. Datenbeschaffung und aufbereitung

<h2>1.1 Datenbeschaffung</h2>

Es müssen einige Bibliotheken hinzugefügt werden. 
Als erstes muss yfinance hinzugefügt werden, diese ermöglich den zugriff auf Aktiendaten zuzugreifen.

In [1]:
!pip install yfinance pandas

Nachdem die benötigten Bibliotheken hinzugefügt wurden müssen diese importiert werden

In [17]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime

Der user soll die möglichkeit bekommen selber die Aktie zu bestimmen, dennoch wird eine Validierung eingebaut die überprüft, ob die Aktie vorhanden ist oder nicht

In [3]:
stock = input("Geben Sie bitte die Aktienkürzel ein: ").upper()
stock_info =  yf.Ticker(stock).info
try:
        if 'shortName' in stock_info and stock_info['shortName']:
            print(f"Die Aktie lautet: {stock_info['shortName']}")
            print(stock_info)
        else:
            raise Exception (f"Die Aktie: {stock} ist nicht vorhanden")
except Exception as err:
        print(err)

Geben Sie bitte die Aktienkürzel ein:  AMZN


Die Aktie lautet: Amazon.com, Inc.
{'address1': '410 Terry Avenue North', 'city': 'Seattle', 'state': 'WA', 'zip': '98109-5210', 'country': 'United States', 'phone': '206 266 1000', 'website': 'https://www.aboutamazon.com', 'industry': 'Internet Retail', 'industryKey': 'internet-retail', 'industryDisp': 'Internet Retail', 'sector': 'Consumer Cyclical', 'sectorKey': 'consumer-cyclical', 'sectorDisp': 'Consumer Cyclical', 'longBusinessSummary': "Amazon.com, Inc. engages in the retail sale of consumer products, advertising, and subscriptions service through online and physical stores in North America and internationally. The company operates through three segments: North America, International, and Amazon Web Services (AWS). It also manufactures and sells electronic devices, including Kindle, Fire tablets, Fire TVs, Echo, Ring, Blink, and eero; and develops and produces media content. In addition, the company offers programs that enable sellers to sell their products in its stores; and pr

Nachdem die Validation steht kann mit der Variable "stock_info" die Daten heruntergeladen werden.
Hierzu werden die Daten vom aktuellen Tag bis vor einem Jahr verwendet.

In [18]:
print("---Metadaten---")
print(stock_info['symbol'])
print(datetime.today().strftime('%Y-%m-%d'))
print((datetime.today().replace(year=datetime.today().year - 1)).strftime('%Y-%m-%d'))
try:
    stock_data = yf.download(
                    stock_info['symbol'],
                    start = (datetime.today().replace(year=datetime.today().year - 1)).strftime('%Y-%m-%d'), 
                    end=datetime.today().strftime('%Y-%m-%d'))
except Exception as err:
    print(f"Das Herunterladen der Datei war nicht erfolgreich: {err}")
print(f"--------------------------Aktiendaten von {stock_info['shortName']}--------------------------")
print(stock_data)
print(f"------------------------------------------------------------------------------------")

[*********************100%%**********************]  1 of 1 completed

---Metadaten---
AMZN
2024-07-08
2023-07-08
--------------------------Aktiendaten von Amazon.com, Inc.--------------------------
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2023-07-10  129.070007  129.279999  125.919998  127.129997  127.129997   
2023-07-11  127.750000  129.770004  127.349998  128.779999  128.779999   
2023-07-12  130.309998  131.259995  128.830002  130.800003  130.800003   
2023-07-13  134.039993  134.669998  132.710007  134.300003  134.300003   
2023-07-14  134.059998  136.649994  134.059998  134.679993  134.679993   
...                ...         ...         ...         ...         ...   
2024-06-28  197.729996  198.850006  192.500000  193.250000  193.250000   
2024-07-01  193.490005  198.300003  192.820007  197.199997  197.199997   
2024-07-02  197.279999  200.429993  195.929993  200.000000  200.000000   
2024-07-03  199.940002  200.029999  196.759995  197.589996

Durch die erlangten können wir festhalten das wir folgende Werte erhalten:
1. Date: Aktienwerte für den tag
2. Open: Der Eröffnungskurs der Aktie 
3. High: Der höchste Kurs der Aktie der am diesem Tag stattgefunden hast
4. Low:  Der niedrigste Kurs der Aktie der am diesem Tag stattgefunden hast
5. Adj Close: Dieser Kurs zeigt den gereinigt Schlusskurs an mit berücksichtigung auf die Dividende, Aktiensplits etc.
6. Volume: Die Anzahl der gehandelten Aktien

In [19]:
stock_data.ffill(inplace=True)

Mit dem Befehl wurde die Datei Bereinigt und passend Convertiert

In [20]:
# Relevante Merkmale auswählen
stock_data['ReturnOnCapital'] = stock_data['Close'].pct_change()
stock_data.dropna(inplace=True)
stock_data

,Open,High,Low,Close,Adj Close,Volume,ReturnOnCapital
Date,,,,,,,
2023-07-11,127.750000,129.770004,127.349998,128.779999,128.779999,49951500,0.012979
2023-07-12,130.309998,131.259995,128.830002,130.800003,130.800003,54022800,0.015686
2023-07-13,134.039993,134.669998,132.710007,134.300003,134.300003,61170900,0.026758
2023-07-14,134.059998,136.649994,134.059998,134.679993,134.679993,54388100,0.002829
2023-07-17,134.559998,135.619995,133.210007,133.559998,133.559998,48450200,-0.008316
...,...,...,...,...,...,...,...
2024-06-28,197.729996,198.850006,192.500000,193.250000,193.250000,76930200,-0.023250
2024-07-01,193.490005,198.300003,192.820007,197.199997,197.199997,41192000,0.020440
2024-07-02,197.279999,200.429993,195.929993,200.000000,200.000000,45600000,0.014199


In [22]:
# Beispiel-Daten
stock_data = pd.DataFrame({
    'Date': pd.date_range(start='2023-07-01', periods=60, freq='D'),
    'Close': np.random.randint(100, 200, size=60)  # Zufällige Schlusskurse
})

# Berechnung der gleitenden Durchschnitte
stock_data['SMA_20'] = stock_data['Close'].rolling(window=20).mean()
stock_data['SMA_50'] = stock_data['Close'].rolling(window=50).mean()
stock_data.head(30)

,Date,Close,SMA_20,SMA_50
0,2023-07-01,106,NaN,NaN
1,2023-07-02,105,NaN,NaN
2,2023-07-03,143,NaN,NaN
3,2023-07-04,176,NaN,NaN
4,2023-07-05,171,NaN,NaN
5,2023-07-06,176,NaN,NaN
6,2023-07-07,135,NaN,NaN
7,2023-07-08,167,NaN,NaN
8,2023-07-09,182,NaN,NaN
9,2023-07-10,139,NaN,NaN
